In [1]:
import os
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from openai import OpenAI
# import anthropic
import gradio as gr
import requests
import json
from typing import List, Dict
from IPython.display import Markdown, display, update_display
from urllib.parse import urljoin
from enum import Enum, auto
from AISystem import formatPrompt, AI, AISystem
import gradio as gr 

load_dotenv()
api_key=os.getenv('DEEPSEEK_API')

if api_key and api_key[:8]=='sk-d07df':
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? ")

MODEL='deepseek-chat'
url = "https://api.deepseek.com"


deepseek_openai = OpenAI(api_key=api_key, base_url=url)

API key looks good so far


In [24]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [25]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})
    messages.append({"role": "user", "content": message})

    print("History is: ")
    print(history)
    print("And messages is: ")
    print(messages)

    stream = deepseek_openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [26]:
gr.ChatInterface(fn=chat).launch()

c:\Users\lilil\miniconda3\envs\llms\Lib\site-packages\gradio\components\chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


History is: 
[]
And messages is: 
[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'Encourage the customer to buy hats if they are unsure what to get."}, {'role': 'user', 'content': 'hi there'}]
History is: 
[['hi there', "Hello! Welcome to our store. How can I assist you today? If you're looking for something specific, we have a fantastic sales event happening right now. Hats are 60% off, and most other items are 50% off. Let me know what you're interested in, and I can help you find the perfect piece! 😊"]]
And messages is: 
[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the custom

In [28]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


History is: 
[]
And messages is: 
[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'Encourage the customer to buy hats if they are unsure what to get.\nIf the customer asks for shoes, you should respond that shoes are not on sale today, but remind the customer to look at hats!\nIf the customer asks for shoes, you should respond that shoes are not on sale today, but remind the customer to look at hats!"}, {'role': 'user', 'content': 'I would like to buy some shoes'}]


In [29]:
def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = deepseek_openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [30]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.
